In [1]:
import pandas as pd, numpy as np
from pyfasttext import FastText
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import csv

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)
train[COMMENT] = train[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")
test[COMMENT] = test[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")

In [3]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['clean'] = 1-train[label_cols].max(axis=1)
train.describe()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean
count,9.585100e+04,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000,95851.000000
mean,4.994359e+11,0.096368,0.010068,0.053301,0.003182,0.049713,0.008492,0.897862
std,2.890136e+11,0.295097,0.099832,0.224635,0.056320,0.217352,0.091762,0.302831
min,2.225664e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473437e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,5.001297e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.501088e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999882e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# pretrain vectors

## youtube comments

In [74]:
def get_youtube_comments_df():
    youtube_gb_comments = pd.read_csv('GBcomments.csv', error_bad_lines = False)
    youtube_us_comments = pd.read_csv('UScomments.csv', error_bad_lines = False)

    youtube_gb_comments[COMMENT].fillna("unknown", inplace=True)
    youtube_gb_comments[COMMENT] = youtube_gb_comments[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")
    youtube_us_comments[COMMENT].fillna("unknown", inplace=True)
    youtube_us_comments[COMMENT] = youtube_us_comments[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")

    youtube_gb_comments.drop(['video_id','likes','replies'],1, inplace=True)
    youtube_us_comments.drop(['video_id','likes','replies'],1, inplace=True)
    return pd.concat([youtube_gb_comments,youtube_us_comments])

## toxicity

In [4]:
def get_toxicity_df():
    toxicity_annotated_comments = pd.read_csv('toxicity_annotated_comments.tsv', sep='\t')
    toxicity_annotations = pd.read_csv('toxicity_annotations.tsv', sep='\t')
    rev_toxicity = toxicity_annotations.drop(['worker_id'],1).groupby(['rev_id']).mean().reset_index()
    toxicity_df = toxicity_annotated_comments.merge(rev_toxicity, left_on='rev_id', right_on='rev_id', how='inner')[['comment','toxicity']]
    # add toxicity to train
    # reshape toxicity into train
    toxicity_df = toxicity_df.rename(index=str, columns={'comment': 'comment_text'})
    toxicity_df['toxic'] = toxicity_df['toxicity'] > 0.5
    toxicity_df['toxic'] = toxicity_df['toxic'].apply(lambda x: 1 if x else 0)
    toxicity_df['id'] = 0
    toxicity_df['severe_toxic'] = 0
    toxicity_df['obscene'] = 0
    toxicity_df['threat'] = 0
    toxicity_df['insult'] = 0
    toxicity_df['identity_hate'] = 0
    toxicity_df[COMMENT].fillna("unknown", inplace=True)
    toxicity_df[COMMENT] = toxicity_df[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace('"',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")
    toxicity_df.drop(['toxicity'],1,inplace=True)
    toxicity_df['clean'] = 1-toxicity_df[label_cols].max(axis=1)
    return toxicity_df

In [ ]:
# TODO wiki comments corpus https://figshare.com/articles/Wikipedia_Talk_Corpus/4264973

# Pretrain comments

In [76]:
youtube_df = get_youtube_comments_df()
toxic_df = get_toxicity_df()

b'Skipping line 113225: expected 4 fields, saw 5\n'
b'Skipping line 158379: expected 4 fields, saw 7\nSkipping line 241590: expected 4 fields, saw 5\nSkipping line 245637: expected 4 fields, saw 7\n'
b'Skipping line 521402: expected 4 fields, saw 5\n'
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


NameError: name 'get_toxic_df' is not defined

In [78]:
#train_comment = train.copy()
#train_comment.drop(label_cols, 1, inplace=True)
#train_comment.drop(['clean'],1, inplace=True)
#train_comment.to_csv("train_comments.csv", columns=['comment_text'], index = False)
all_comment = pd.concat([train,test,youtube_df,toxic_df])
all_comment.to_csv('all_comments.csv', columns=['comment_text'], index = False)

In [34]:
#all_wiki_youtube_comment = pd.concat([train_comment,test,youtube_df])
#all_wiki_youtube_comment.to_csv('all_wiki_youtube_comments.csv', columns=['comment_text'], index = False)

In [98]:
model = FastText()
model.skipgram(input='all_comments.csv', output='comments_min2_max5_word2_epoch_10', minn='2', maxn='5', wordNgrams='2', epoch='1', dim='150')
#model.skipgram(input='all_wiki_youtube_comments.csv', output='awy_comments_skipgram', minn='1', maxn='5', wordNgrams='3', epoch='1')
#model = FastText()
del model
#model.skipgram(input='train_comments.csv', output='train_comments_skipgram_1_1_2', minn='1', maxn='1', wordNgrams='2')
#del model
#model1 = FastText()
#model1.cbow(input='train_comments.csv', output='train_comments_cbow', minn='1', maxn='5', wordNgrams='2')
#del model1

# train model

In [5]:
train_df = None
val_df = None
#toxicity_df = get_toxicity_df() 
for label in label_cols:
    train['label'] = ""    
    train_df, val_df = train_test_split(train, test_size=0.2, random_state=1)
    # merge toxicity before training
    #train_df = pd.concat([train_df, toxicity_df])
    train_df.loc[(train_df[label]==1), 'label'] = "__label__" + label + " "
    train_df.loc[(train_df[label]==0), 'label'] = "__label__clean" + " "
    val_df.loc[(val_df[label]==1), 'label'] = "__label__" + label + " "
    val_df.loc[(val_df[label]==0), 'label'] = "__label__clean" + " "
    train_df.to_csv("fasttext_train_"+label+".csv", columns=['label', 'comment_text'], index=False)    
    train.to_csv("fasttext_train_all_"+label+".csv", columns=['label', 'comment_text'], index=False)    

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
def model_name(target, param):
    name = target
    for key in param.keys():
        name += "_"+key+str(param[key])        
    return name

In [7]:
def train_model(param, target, all_sample = False):
    model = FastText()    
    if all_sample == False:
        model.supervised(input='fasttext_train_'+target+'.csv', output='parameter_search', epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"], wordNgrams=param["wordNgrams"], dim=param["dim"], neg=param["neg"], pretrainedVectors=param["pretrainedVectors"])
        #model.supervised(input='fasttext_train_'+target+'.csv', output='parameter_search', epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"], wordNgrams=param["wordNgrams"], dim=param["dim"], neg=param["neg"])
    else:
        model.supervised(input='fasttext_train_all_'+target+'.csv', output='parameter_search', epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"], wordNgrams=param["wordNgrams"], dim=param["dim"], neg=param["neg"], pretrainedVectors=param["pretrainedVectors"])
        #model.supervised(input='fasttext_train_all_'+target+'.csv', output='parameter_search', epoch=param["epoch"], minn=param["minn"], maxn=param["maxn"], wordNgrams=param["wordNgrams"], dim=param["dim"], neg=param["neg"])        
    return model

In [8]:
def calculate_log_loss(model, target):
    predict_probas = model.predict_proba(val_df[COMMENT],k=2) 
    pred = []
    for predict in predict_probas:
        pred_prob = 0
        for label, prob in predict:
            if label == target:         
                pred_prob = prob
        pred.append(pred_prob) 
    return log_loss(val_df[target],pred)

In [10]:
import random
param_grid = {}
param_grid["minn"] = [1,2]
param_grid["maxn"] = [2,3,4,5]
param_grid["epoch"] = [5,6,7,8,10,15]
param_grid["lr"] = [0.1,0.5,1]
param_grid["wordNgrams"] =[1,2,3,4,5]
param_grid["dim"] =[100,150,200]
targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

log_loss_df_columns = ['target','log_loss','model_name']
for key in param_grid.keys():
    log_loss_df_columns.append(key)

log_loss_df = None
try:
    log_loss_df = pd.read_csv('parameter_random_search.csv')    
except:
    print("parameter_random_search.csv not found")
    log_loss_df = pd.DataFrame(columns=log_loss_df_columns)

tested_params = log_loss_df['model_name'].tolist()

for target in targets:
    print("parameter search",target)    
    max_test = len(tested_params) + 10
    while len(tested_params) < max_test:        
        param = random.choice(list(ParameterGrid(param_grid)))                     
        test_model_name = model_name(target, param)
        if test_model_name in tested_params:
            continue
        else:
            tested_params.append(test_model_name)
            
        model = train_model(param, target)
        score = calculate_log_loss(model, target)
        
        log_loss_df_row = {}
        log_loss_df_row['target'] = target
        log_loss_df_row['log_loss'] = score
        log_loss_df_row['model_name'] = test_model_name
        for key in param.keys():
            log_loss_df_row[key] = param[key]
        log_loss_df = log_loss_df.append(log_loss_df_row, ignore_index=True)        
        log_loss_df.to_csv("parameter_random_search.csv", index=False)    
        print(score, param)   

parameter search toxic
parameter search severe_toxic
parameter search obscene
parameter search threat
parameter search insult
parameter search identity_hate


In [13]:
param_grid = {}
param_grid["minn"] = [2]
param_grid["maxn"] = [5]
param_grid["epoch"] = [1]
param_grid["lr"] = [0.0001]
param_grid["wordNgrams"] =[2]
param_grid["dim"] = [200]
param_grid["neg"] = [0]
param_grid["pretrainedVectors"]=['wiki_comments/inputtest_article_comments_nosurf_dim200.vec']
targets = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
#targets = ['toxic']
#comments_min2_max5_word2_epoch_10

log_loss_df_columns = ['target','log_loss']
for key in param_grid.keys():
    log_loss_df_columns.append(key)

log_loss_df = pd.DataFrame(columns=log_loss_df_columns)

for target in targets:
    print("parameter search",target)    
    for param in list(ParameterGrid(param_grid)):                     
        model = train_model(param, target)
        score = calculate_log_loss(model, target)
        
        log_loss_df_row = {}
        log_loss_df_row['target'] = target
        log_loss_df_row['log_loss'] = score
        for key in param.keys():
            log_loss_df_row[key] = param[key]
        log_loss_df = log_loss_df.append(log_loss_df_row, ignore_index=True)        
        log_loss_df.to_csv("parameter_grid_search.csv", index=False)    
        print(score, param)    

parameter search toxic
0.187266459037 {'maxn': 5, 'lr': 0.0001, 'epoch': 1, 'minn': 2, 'dim': 200, 'wordNgrams': 2, 'pretrainedVectors': 'wiki_comments/inputtest_article_comments_nosurf_dim200.vec', 'neg': 0}
parameter search severe_toxic
0.0405078810351 {'maxn': 5, 'lr': 0.0001, 'epoch': 1, 'minn': 2, 'dim': 200, 'wordNgrams': 2, 'pretrainedVectors': 'wiki_comments/inputtest_article_comments_nosurf_dim200.vec', 'neg': 0}
parameter search obscene
0.108811285734 {'maxn': 5, 'lr': 0.0001, 'epoch': 1, 'minn': 2, 'dim': 200, 'wordNgrams': 2, 'pretrainedVectors': 'wiki_comments/inputtest_article_comments_nosurf_dim200.vec', 'neg': 0}
parameter search threat
0.0172615014757 {'maxn': 5, 'lr': 0.0001, 'epoch': 1, 'minn': 2, 'dim': 200, 'wordNgrams': 2, 'pretrainedVectors': 'wiki_comments/inputtest_article_comments_nosurf_dim200.vec', 'neg': 0}
parameter search insult
0.128582444098 {'maxn': 5, 'lr': 0.0001, 'epoch': 1, 'minn': 2, 'dim': 200, 'wordNgrams': 2, 'pretrainedVectors': 'wiki_comments

In [14]:
log_loss_df.groupby('target')['log_loss'].min().sum()/6

0.087380308815817334

In [15]:
log_loss_df.groupby('target')['log_loss'].min()

target
identity_hate    0.041852
insult           0.128582
obscene          0.108811
severe_toxic     0.040508
threat           0.017262
toxic            0.187266
Name: log_loss, dtype: float64

In [16]:
# label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
log_loss_df[log_loss_df['target']=='identity_hate'].sort_values('log_loss',ascending=True)

,target,log_loss,minn,lr,epoch,maxn,wordNgrams,dim,pretrainedVectors,neg
5,identity_hate,0.041852,2,0.0001,1,5,2,200,wiki_comments/inputtest_article_comments_nosur...,0


# Predict

In [70]:
param = {}
target='toxic'
param['toxic'] = {'maxn': 1, 'wordNgrams': 2, 'lr': 0.5, 'minn': 1, 'epoch': 5, 'dim': 100}
model = train_model(param[target], target, all_sample = True)

In [13]:
comment_ids = []
comments = []
for index, row in test.iterrows():
    comments.append(row['comment_text'])
    comment_ids.append(row['id'])

predict_probas = {}
param = {}
param['toxic'] = {'maxn': 5, 'wordNgrams': 3, 'lr': 0.5, 'minn': 1, 'epoch': 8, 'dim': 100}
param['severe_toxic'] = {'maxn': 5, 'wordNgrams': 4, 'lr': 0.5, 'minn': 2, 'epoch': 5, 'dim': 100}
param['obscene'] = {'maxn': 5, 'wordNgrams': 1, 'lr': 0.1, 'minn': 1, 'epoch': 8, 'dim': 100}
param['threat'] = {'maxn': 4, 'wordNgrams': 5, 'lr': 1, 'minn': 1, 'epoch': 10, 'dim': 100}
param['insult'] = {'maxn': 5, 'wordNgrams': 3, 'lr': 0.5, 'minn': 1, 'epoch': 5, 'dim': 100}
param['identity_hate'] = {'maxn': 5, 'wordNgrams': 3, 'lr': 1, 'minn': 2, 'epoch': 6, 'dim': 100}

for target in label_cols:
    model = train_model(param[target], target, all_sample = True)
    print(model.labels)
    predict_probas[target] = model.predict_proba(comments,k=2)   
    del model

['clean', 'toxic']
['clean', 'severe_toxic']
['clean', 'obscene']
['clean', 'threat']
['clean', 'insult']
['clean', 'identity_hate']


In [21]:
import csv
with open('submit.csv', "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])
    csv_rows = []
    for index, comment_id in enumerate(comment_ids):
        #print('processing',index)
        prob_dict = {}        
        #print(comments[index])
        for label in label_cols:       
            #print(predict_probas[label][index])            
            for label_predict, prob in predict_probas[label][index]:
                if label_predict != 'clean':    
                    prob_dict[label] = prob        
        csv_row=[]
        csv_row.append(comment_id)
        
        # severe_toxic -> toxic trick                
        #if 'severe_toxic' in prob_dict and 'toxic' in prob_dict and prob_dict['severe_toxic'] > prob_dict['toxic']:            
        #    prob_dict['toxic'] = prob_dict['severe_toxic']            
        
        for label in label_cols:
            if label in prob_dict:
                csv_row.append(prob_dict[label])
            else:
                csv_row.append(0)
                #print('no prediction:',index,'comment:',comments[index])
                #csv_row.append(prob_dict[label])
        csv_rows.append(csv_row)    
    writer.writerows(csv_rows)